# Chuck's Playground

In [63]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from PyPDF2 import PdfReader
from io import BytesIO
import re
import json
import os
from datetime import datetime
from datamanager import ResponseDB
import config
# Configuration constants
# Field mappings for form responses
# Add/modify fields here when form questions change
FIELD_MAPPINGS = config.FIELD_MAPPINGS

def sanitize_field_name(field_name):
    """Convert field names to lowercase with underscores instead of spaces."""
    return field_name.lower().replace(" ", "_")

def setup_directories():
    """Create necessary directories for storing responses and attachments."""
    for directory in [config.OUTPUT_DIR, config.ATTACHMENT_DIR]:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Created directory: {directory}")

def initialize_google_services():
    """Initialize and return Google Sheets and Drive services."""
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        config.CREDENTIALS_FILE,
        config.SCOPES
    )
    
    sheets_client = gspread.authorize(creds)
    drive_service = build('drive', 'v3', credentials=creds)
    
    return sheets_client, drive_service

def download_file_from_drive(service, file_id, mime_type='application/pdf'):
    """
    Download a file from Google Drive using the Drive API.
    
    Args:
        service: Google Drive service instance
        file_id: ID of the file to download
        mime_type: Expected MIME type of the file
    
    Returns:
        BytesIO object containing the file content
    """
    try:
        # Verify file type
        file_metadata = service.files().get(fileId=file_id, fields='mimeType').execute()
        if file_metadata['mimeType'] != mime_type:
            raise ValueError(f"File is not the expected type. Expected: {mime_type}, Got: {file_metadata['mimeType']}")

        # Download file
        request = service.files().get_media(fileId=file_id)
        file_content = BytesIO()
        downloader = MediaIoBaseDownload(file_content, request)
        
        # Show download progress
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if status:
                print(f"Download Progress: {int(status.progress() * 100)}%")
                
        file_content.seek(0)
        return file_content.read()
        
    except Exception as e:
        print(f"Error downloading file: {str(e)}")
        raise

def extract_text_from_pdf(pdf_content):
    """Extract text content from a PDF file."""
    try:
        with BytesIO(pdf_content) as pdf_file:
            reader = PdfReader(pdf_file)
            return '\n'.join(
                page.extract_text() for page in reader.pages
                if page.extract_text()
            )
    except Exception as e:
        return f"Error processing PDF: {str(e)}"

def extract_file_id(url):
    """Extract file ID from various Google Drive URL formats."""
    patterns = [
        r'/file/d/([^/]+)',
        r'id=([^&]+)',
        r'open\?id=([^&]+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def process_attachment(drive_service, url, field_config, response_id):
    """
    Process an attachment field from the form response.
    
    Args:
        drive_service: Google Drive service instance
        url: URL of the attachment
        field_config: Configuration for this field
        response_id: Unique identifier for this response
        
    Returns:
        dict containing processed attachment information
    """
    result = {
        'original_url': url,
        'local_path': None,
        'extracted_text': None,
        'error': None
    }
    
    try:
        file_id = extract_file_id(url)
        if not file_id:
            raise ValueError("Could not extract file ID from URL")
            
        # Download file
        content = download_file_from_drive(
            drive_service, 
            file_id, 
            f"application/{field_config['format']}"
        )
        
        # Save file
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{response_id}_{timestamp}.{field_config['format']}"
        filepath = os.path.join(config.ATTACHMENT_DIR, filename)
        
        with open(filepath, 'wb') as f:
            f.write(content)
        
        result['local_path'] = filepath
        
        # Extract text if configured
        if field_config.get('extract_text'):
            result['extracted_text'] = extract_text_from_pdf(content)
            
    except Exception as e:
        result['error'] = str(e)
        
    return result

In [65]:
setup_directories()
sheets_client, drive_service = initialize_google_services()
db = ResponseDB()

# Get form responses
sheet = sheets_client.open(config.SPREADSHEET_NAME).sheet1

In [66]:
responses = sheet.get_all_records()
print(responses)

[{'Timestamp': '2/22/2025 13:20:38', 'Email Address': 'pandudparadox@gmail.com', 'Phone Number': 'wa+6287883067863', 'Resume/CV': 'https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu', 'Nama Lengkap': 'Pandu Dhaulagiri', 'Tanggal Lahir': '3/21/2001', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Diinginkan': 'Programmer', 'Domisili': ''}, {'Timestamp': '2/22/2025 13:34:51', 'Email Address': 'remorevovel@gmail.com', 'Phone Number': 'wa+62811809780', 'Resume/CV': 'https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA', 'Nama Lengkap': 'Markus Andreas', 'Tanggal Lahir': '1/1/1999', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Diinginkan': 'Engineer', 'Domisili': ''}, {'Timestamp': '2/22/2025 15:29:08', 'Email Address': 'pandudparadox@gmail.com', 'Phone Number': 'wa+62811809785', 'Resume/CV': 'https://drive.google.com/open?id=1ZoA_e5_hgBd36CJwCGPvBJlmXe6t_rqP', 'Nama Lengkap': 'Artes Paradox', 'Tanggal Lahir': '2/3/1999', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Di

In [46]:
from gspread.utils import GridRangeType

list_of_lists = sheet.get(return_type=GridRangeType.ListOfLists)
responses = [dict(zip(list_of_lists[0], row)) for row in list_of_lists[1:]]

print(responses)

[{'Timestamp': '2/22/2025 13:20:38', 'Email Address': 'pandudparadox@gmail.com', 'Phone Number': 'wa+6287883067863', 'Resume/CV': 'https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu', 'Nama Lengkap': 'Pandu Dhaulagiri', 'Tanggal Lahir': '3/21/2001', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Diinginkan': 'Programmer'}, {'Timestamp': '2/22/2025 13:34:51', 'Email Address': 'remorevovel@gmail.com', 'Phone Number': 'wa+62811809780', 'Resume/CV': 'https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA', 'Nama Lengkap': 'Markus Andreas', 'Tanggal Lahir': '1/1/1999', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Diinginkan': 'Engineer'}, {'Timestamp': '2/22/2025 15:29:08', 'Email Address': 'pandudparadox@gmail.com', 'Phone Number': 'wa+62811809785', 'Resume/CV': 'https://drive.google.com/open?id=1ZoA_e5_hgBd36CJwCGPvBJlmXe6t_rqP', 'Nama Lengkap': 'Artes Paradox', 'Tanggal Lahir': '2/3/1999', 'Jenis Kelamin': 'Laki-Laki', 'Posisi Yang Diinginkan': 'Courier'}]


ok so like we can use GridRangeType for flexible header config

In [70]:
print(f"📥 Found {len(responses)} responses to process...")

# Track processed and skipped responses
processed_count = 0
skipped_count = 0

# Process each response
for idx, response in enumerate(responses, 1):
    phone_number = str(response.get('Phone Number', ''))  # Convert to string for consistency
    timestamp = response.get('Timestamp', '')
    
    # Check if response already exists

    if db.check_duplicate(phone_number, timestamp):
        print(f"⏭️ Skipping duplicate response {idx}/{len(responses)} "
                f"(Phone: {phone_number}, Timestamp: {timestamp})")
        skipped_count += 1
        continue

    response_id = response.get('id', datetime.now().strftime('%Y%m%d_%H%M%S'))
    processed_response = {sanitize_field_name(k): v for k, v in response.items()}
    
    # Add extra fields with default values
    processed_response["questions"] = ""
    processed_response["answers"] = ""
    processed_response["eval"] = ""
    processed_response["score"] = 0
    
    # Process special fields
    for field_name, field_config in FIELD_MAPPINGS.items():
        if field_name in response and response[field_name]:
            if field_config['type'] == 'attachment':
                processed_response[field_name] = process_attachment(
                    drive_service,
                    response[field_name],
                    field_config,
                    response_id
                )
    
    # Save to database
    db.upsert_response(processed_response)
    print(f"✅ Processed response {idx}/{len(responses)} (ID: {response_id})")
    processed_count += 1

print(f"\n🎉 Processing complete:")
print(f"   ✅ Successfully processed: {processed_count} responses")
print(f"   ⏭️ Skipped duplicates: {skipped_count} responses")
print(f"💾 Database saved to: {config.DATABASE_FILE}")

📥 Found 3 responses to process...
Checking for Phone: wa+6287883067863, timestamp: 2/22/2025 13:20:38
Found matches: 1
First match: {'timestamp': '2/22/2025 13:20:38', 'email_address': 'pandudparadox@gmail.com', 'phone_number': 'wa+6287883067863', 'resume/cv': 'https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu', 'nama_lengkap': 'Pandu Dhaulagiri', 'tanggal_lahir': '3/21/2001', 'jenis_kelamin': 'Laki-Laki', 'posisi_yang_diinginkan': 'Programmer', 'domisili': '', 'questions': '', 'answers': '', 'eval': '', 'score': 0, 'Resume/CV': {'original_url': 'https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu', 'local_path': 'attachments/20250313_011017_20250313_011021.pdf', 'extracted_text': 'Pandu\n \nDhaula giri\n \n(Ar s\n \nParado x)\n \npandudparadox@gmail.com\n \n❖\n \n(+62)\n \n859-6597-5929\n \n❖\n \nDiscord\n \n@pandu.paradox\n \n❖\n \nGithub\n \n@Iteranya\n \n❖\n \nlinkedIn\n \n@pandu-dhaulagiri\n \n \nFEATS\n \nOF\n \nENGINEERING\n \n \nCr eated\n \nAn\n

In [68]:
# Track processed and skipped responses
processed_count = 0
skipped_count = 0

for idx, response in enumerate(responses, 1):
    phone_number = str(response.get('Phone Number', ''))  # Convert to string for consistency
    timestamp = response.get('Timestamp', '')
    
    # Check if response already exists

    if db.check_duplicate(phone_number, timestamp):
        print(f"⏭️ Skipping duplicate response {idx}/{len(responses)} "
                f"(Phone: {phone_number}, Timestamp: {timestamp})")
        skipped_count += 1
        continue

    processed_response = {sanitize_field_name(k): v for k, v in response.items()}

    for key, value in processed_response.items():
        print(f"{key}: {value}")

Checking for Phone: wa+6287883067863, timestamp: 2/22/2025 13:20:38
Found matches: 0
timestamp: 2/22/2025 13:20:38
email_address: pandudparadox@gmail.com
phone_number: wa+6287883067863
resume/cv: https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu
nama_lengkap: Pandu Dhaulagiri
tanggal_lahir: 3/21/2001
jenis_kelamin: Laki-Laki
posisi_yang_diinginkan: Programmer
domisili: 
Checking for Phone: wa+62811809780, timestamp: 2/22/2025 13:34:51
Found matches: 0
timestamp: 2/22/2025 13:34:51
email_address: remorevovel@gmail.com
phone_number: wa+62811809780
resume/cv: https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA
nama_lengkap: Markus Andreas
tanggal_lahir: 1/1/1999
jenis_kelamin: Laki-Laki
posisi_yang_diinginkan: Engineer
domisili: 
Checking for Phone: wa+62811809785, timestamp: 2/22/2025 15:29:08
Found matches: 0
timestamp: 2/22/2025 15:29:08
email_address: pandudparadox@gmail.com
phone_number: wa+62811809785
resume/cv: https://drive.google.com/open?id=1ZoA_e

In [67]:
# Track processed and skipped responses
processed_count = 0
skipped_count = 0

for idx, response in enumerate(responses, 1):
    phone_number = str(response.get('Phone Number', ''))  # Convert to string for consistency
    timestamp = response.get('Timestamp', '')
    
    # Check if response already exists

    if db.check_duplicate(phone_number, timestamp):
        print(f"⏭️ Skipping duplicate response {idx}/{len(responses)} "
                f"(Phone: {phone_number}, Timestamp: {timestamp})")
        skipped_count += 1
        continue

    for key, value in response.items():
        print(f"{key}: {value}")

Checking for Phone: wa+6287883067863, timestamp: 2/22/2025 13:20:38
Found matches: 0
Timestamp: 2/22/2025 13:20:38
Email Address: pandudparadox@gmail.com
Phone Number: wa+6287883067863
Resume/CV: https://drive.google.com/open?id=1nyksXNCdhRHAL7hLJrKaZbCb1ZBhXjAu
Nama Lengkap: Pandu Dhaulagiri
Tanggal Lahir: 3/21/2001
Jenis Kelamin: Laki-Laki
Posisi Yang Diinginkan: Programmer
Domisili: 
Checking for Phone: wa+62811809780, timestamp: 2/22/2025 13:34:51
Found matches: 0
Timestamp: 2/22/2025 13:34:51
Email Address: remorevovel@gmail.com
Phone Number: wa+62811809780
Resume/CV: https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA
Nama Lengkap: Markus Andreas
Tanggal Lahir: 1/1/1999
Jenis Kelamin: Laki-Laki
Posisi Yang Diinginkan: Engineer
Domisili: 
Checking for Phone: wa+62811809785, timestamp: 2/22/2025 15:29:08
Found matches: 0
Timestamp: 2/22/2025 15:29:08
Email Address: pandudparadox@gmail.com
Phone Number: wa+62811809785
Resume/CV: https://drive.google.com/open?id=1ZoA_e

## Learning tinydb

In [56]:
from tinydb import TinyDB, Query
import config
import json

db = TinyDB("responses_db.json")

Q = Query()

results = db.search(Q.questions == "")
for row in results:
    print(row.get('Resume/CV').get('extracted_text'))

Pandu
 
Dhaula giri
 
(Ar s
 
Parado x)
 
pandudparadox@gmail.com
 
❖
 
(+62)
 
859-6597-5929
 
❖
 
Discord
 
@pandu.paradox
 
❖
 
Github
 
@Iteranya
 
❖
 
linkedIn
 
@pandu-dhaulagiri
 
 
FEATS
 
OF
 
ENGINEERING
 
 
Cr eated
 
An
 
Embedded
 
Visual
 
No vel
 
Engine
 
from
 
Scratch
 
 
 
           
 
 
No v.
 
2024
 
–
 
Pr esent
 
Solo
 
De veloper
 
 
          
 
        
 
                    
 
                                
 
        
 
        
 
 
 
   
 
  
Python
 
and
 
Java
 
▪
 
Created
 
an
 
entire
 
Visual
 
No vel
 
Game
 
Engine
 
from
 
scratc h
 
and
 
runs
 
it
 
in
 
Minecraft.
 
▪
 
Skills
 
Lear ned/Impro ved
 
o
 
Dee p
 
Understanding
 
of
 
working
 
with
 
unfamiliar
 
codebase
 
that
 
is
 
obfuscated
 
like
 
Minecraft
 
o
 
The
 
creation
 
of
 
a
 
Pseudo-DSL,
 
essentially
 
a
 
new
 
prog ramming
 
languag e
 
to
 
create
 
scripts
 
 
o
 
Creation
 
an
 
Embedded
 
Visual
 
No vel
 
Game
 
Engine
 
In
 
A
 
Hostile
 
En vironment
 
o
 
Creation

In [32]:
print(results[1])

{'timestamp': '2/22/2025 13:34:51', 'email_address': 'remorevovel@gmail.com', 'phone_number': 'wa+62811809780', 'resume/cv': 'https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA', 'nama_lengkap': 'Markus Andreas', 'tanggal_lahir': '1/1/1999', 'jenis_kelamin': 'Laki-Laki', 'posisi_yang_diinginkan': 'Engineer', 'domisili': '', 'questions': '', 'answers': '', 'eval': '', 'score': 0, 'Resume/CV': {'original_url': 'https://drive.google.com/open?id=18v19AudL_nrXRlmu5WNUCW41GlTl0aNA', 'local_path': 'attachments/20250312_232047_20250312_232056.pdf', 'extracted_text': '', 'error': None}}
